In [1]:
import pandas as pd

### Create inverse rates

In [64]:
df = pd.read_csv('testData.csv')
# append copy of dataframe to create inverse rates
df2 = df.copy()
df2.Rate = list(map(lambda x: float(1/x), df2.Rate))
df2 = df2.rename(columns={'Numerator':'Denumerator','Denumerator':'Numerator'})

df = (df
        .append(df2)
        .drop(columns=['Unnamed: 0'])
        .reset_index()
        .drop(columns=['index'])
     )

(103, 7)
(206, 6)


### Create possible pairs in triangular arbitrage

In [66]:
pairList = []
initialCoin = 'BTC'
# for every denumerator that has BTC as numerator
for i in list(df.Denumerator[df['Numerator'] == initialCoin]):
    # for every denumerator that has i as numerator
    for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != initialCoin) & (df['Denumerator'] != initialCoin)]))):
        if j in list(df.Denumerator[df['Numerator'] == initialCoin]):
            pairList.append([i,j])

### Data preparation function

In [2]:
def prepareData(df, initialCoin = 'BTC'):
    # INVERSE RATES CREATION
    # append copy of dataframe to create inverse rates
    df2 = df.copy()
    df2.Rate = list(map(lambda x: float(1/x), df2.Rate))
    df2 = df2.rename(columns={'Numerator':'Denumerator','Denumerator':'Numerator'})

    df = (df
            .append(df2)
            .drop(columns=['Unnamed: 0'])
            .reset_index()
            .drop(columns=['index'])
         )
    
    # POSSIBLE PAIRS CREATION
    pairList = []
    # for every denumerator that has BTC as numerator
    for i in list(df.Denumerator[df['Numerator'] == initialCoin]):
        # for every denumerator that has i as numerator
        for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != initialCoin) & (df['Denumerator'] != initialCoin)]))):
            if j in list(df.Denumerator[df['Numerator'] == initialCoin]):
                pairList.append([i,j])
    
    return df, pairList

### Real data arbitrage

In [8]:
def triangularArbitrage(df, inputSet, firstTransaction, secondTransaction, fee):
    # first transaction
    output1 = inputSet[0]/df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)].values[0]
    output1 = output1 - fee*output1
    
    # second transaction
    output2 = output1/df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)].values[0]
    output2 = output2 - fee*output2
    
    # third transaction
    output3 = output2/df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])].values[0]
    output2 = output2 - fee*output2
    
#     if float(output3 - inputSet[0]) > 0:
#         print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', float(output3 - inputSet[0]), inputSet[1])
    
    return float(output3 - inputSet[0])

# inputSet = (1, 'BTC')
# %time triangularArbitrage(inputSet, 'LTC', 'ETH', fee = 0.0004)

In [90]:
# read data
df = pd.read_csv('testData.csv')
# transform dataframe
initialCoin = 'USDT'
df, pairList = prepareData(df, initialCoin = initialCoin)
baseCoins = list(df[:103].Denumerator.unique())
print('Possible initial coins:', baseCoins)

Possible initial coins: ['USDT', 'BTC', 'PAX', 'ETH', 'USDC']


In [94]:
%%time
baseCoins = ['USDT', 'BTC', 'PAX', 'ETH', 'USDC']
for coin in baseCoins:
    df = pd.read_csv('testData.csv')
    df, pairList = prepareData(df, initialCoin = coin)
    suma = 0
    n = 1
    btcQuantity = 1
    for i in pairList:
        add = triangularArbitrage((btcQuantity, coin), i[0], i[1], fee = 0.0004)
        if add > 0:
            suma += add
            btcQuantity += add
            n += 1
    print(f'Obtained: {round(suma * 20143.63, 2)} PLN by {n} triangular transactions using cummulated {coin} asset')

Obtained: 6536.04 PLN by 40 triangular transactions using cummulated USDT asset
Obtained: 5781.32 PLN by 45 triangular transactions using cummulated BTC asset
Obtained: 317.42 PLN by 5 triangular transactions using cummulated PAX asset
Obtained: 3546.26 PLN by 37 triangular transactions using cummulated ETH asset
Obtained: 86.73 PLN by 3 triangular transactions using cummulated USDC asset
Wall time: 1.82 s


In [9]:
def findArbitrage():
    baseCoins = ['USDT', 'BTC', 'PAX', 'ETH', 'USDC']
    for coin in baseCoins:
        df = pd.read_csv('testData.csv')
        df, pairList = prepareData(df, initialCoin = coin)
        suma = 0
        n = 1
        btcQuantity = 1
        for i in pairList:
            add = triangularArbitrage(df, (btcQuantity, coin), i[0], i[1], fee = 0.0004)
            if add > 0:
                suma += add
                btcQuantity += add
                n += 1
        print(f'Obtained: {round(suma * 20143.63, 2)} PLN by {n} triangular transactions using cummulated {coin} asset')

In [10]:
findArbitrage()

Obtained: 6536.04 PLN by 40 triangular transactions using cummulated USDT asset
Obtained: 5781.32 PLN by 45 triangular transactions using cummulated BTC asset
Obtained: 317.42 PLN by 5 triangular transactions using cummulated PAX asset
Obtained: 3546.26 PLN by 37 triangular transactions using cummulated ETH asset
Obtained: 86.73 PLN by 3 triangular transactions using cummulated USDC asset


In [17]:
class Arbitrage:
    def __init__(self):
        pass
    
    def prepareData(self, df, initialCoin = 'BTC'):
        # INVERSE RATES CREATION
        # append copy of dataframe to create inverse rates
        df2 = df.copy()
        df2.Rate = list(map(lambda x: float(1/x), df2.Rate))
        df2 = df2.rename(columns={'Numerator':'Denumerator','Denumerator':'Numerator'})

        df = (df
                .append(df2)
                .drop(columns=['Unnamed: 0'])
                .reset_index()
                .drop(columns=['index'])
             )

        # POSSIBLE PAIRS CREATION
        pairList = []
        # for every denumerator that has BTC as numerator
        for i in list(df.Denumerator[df['Numerator'] == initialCoin]):
            # for every denumerator that has i as numerator
            for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != initialCoin) & (df['Denumerator'] != initialCoin)]))):
                if j in list(df.Denumerator[df['Numerator'] == initialCoin]):
                    pairList.append([i,j])

        return df, pairList
    
    
    def triangularArbitrage(self, df, inputSet, firstTransaction, secondTransaction, fee):
        # first transaction
        output1 = inputSet[0]/df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)].values[0]
        output1 = output1 - fee*output1

        # second transaction
        output2 = output1/df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)].values[0]
        output2 = output2 - fee*output2

        # third transaction
        output3 = output2/df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])].values[0]
        output2 = output2 - fee*output2

    #     if float(output3 - inputSet[0]) > 0:
    #         print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', float(output3 - inputSet[0]), inputSet[1])

        return float(output3 - inputSet[0])
    
    
    def findArbitrage(self):
        baseCoins = ['USDT', 'BTC', 'PAX', 'ETH', 'USDC']
        for coin in baseCoins:
            df = pd.read_csv('testData.csv')
            df, pairList = self.prepareData(df = df, initialCoin = coin)
            suma = 0
            n = 1
            btcQuantity = 1
            for i in pairList:
                add = self.triangularArbitrage(df = df, inputSet = (btcQuantity, coin), firstTransaction = i[0], secondTransaction = i[1], fee = 0.0004)
                if add > 0:
                    suma += add
                    btcQuantity += add
                    n += 1
            print(f'Obtained: {round(suma * 20143.63, 2)} PLN by {n} triangular transactions using cummulated {coin} asset')

In [18]:
kA = Arbitrage()

In [19]:
kA.findArbitrage()

Obtained: 6536.04 PLN by 40 triangular transactions using cummulated USDT asset
Obtained: 5781.32 PLN by 45 triangular transactions using cummulated BTC asset
Obtained: 317.42 PLN by 5 triangular transactions using cummulated PAX asset
Obtained: 3546.26 PLN by 37 triangular transactions using cummulated ETH asset
Obtained: 86.73 PLN by 3 triangular transactions using cummulated USDC asset
